### **Звіт**

**Бригада 2** 

##### **Мета:**

Обчислення критерію G(x_h) (індекс Джіні) для кожної
ознаки обраного датасету та вибір ознаки x*_h, яка мінімізує G(x_h) задля визначення найбільш інформативної ознаки для побудови рішення. 

##### **Хід роботи:**

1. Імпорт бібліотек.

In [2]:
import numpy as np
import pandas as pd

2. Завантаження та підготовка даних:

> - Завантаження датасету з GitHub, створення власної таблиці df з цими даними.  

In [3]:
url = "https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv"
df = pd.read_csv(url)

> - Обираємо стовпці таблиці, що відповідають за потрібні нам ознаки:

 *Pclass - клас пасажира; Sex - стать пасажира; SibSp - кількість братів, сестер або подружжя на борту «Титаніка»; Parch - кількість батьків або дітей на борту «Титаніка».*

In [4]:
df = df[['Pclass', 'Sex', 'SibSp', 'Parch', 'Survived']].dropna()

> - Створюємо матрицю ознак та вектор міток.

In [5]:
T = df[['Pclass', 'Sex', 'SibSp', 'Parch']]  
y = df['Survived']  

> - Перетворюємо дані матриці ознак та вектора міток в рядковий формат, аби представити ознаки як категоріальні.

In [6]:
T_str = T.map(str)
y_str = y.astype(str)

3. Функції для обчислення G(x_h) та індексу Джіні:

> - Оголошуємо функцію *gini_index(subset)*, яка: рахує кількість елементів у підмножині, облислює частки класів, повертає індекс Джині.

In [7]:
def gini_index(subset):
    cardinality = len(subset)
    if cardinality == 0:
        return 0
    counts = subset.value_counts()
    p_j = counts / cardinality
    return 1 - np.sum(p_j**2)

> - Оголошуємо функцію *feature_gscore(data, labels, feature)*, яка: для кожного унікального значення ознаки формує підмножину; обчислює її індекс Джіні; підсумовує зважені значення, отримує G(x_h).

In [8]:
def feature_gscore(data, labels, feature):
    values = data[feature].unique()
    N = len(data)
    G = 0
    for val in values:
        subset_idx = data[feature] == val
        subset_labels = labels[subset_idx]
        G += (len(subset_labels) / N) * gini_index(subset_labels)
    return G

> - Оголошуємо функцію *feature_details(data, labels, feature)*, яка: для кожного значення ознаки повертає кількість прикладів, індекс Джіні, розподіл класів.

In [9]:
def feature_details(data, labels, feature):
    values = data[feature].dropna().unique()
    details = {}
    for val in values:
        subset_idx = data[feature] == val
        subset_labels = labels[subset_idx]
        details[val] = {
            "count": len(subset_labels),
            "gini": gini_index(subset_labels),
            "class_dist": subset_labels.value_counts().to_dict()
        }
    return details

> - Оголошуємо функцію *best_feature(data, labels)*, яка: обчислює G(x_h) для всіх ознак; знаходить ознаку з мінімальним значенням G(x_h); повертає її назву і всі результати для порівняння.

In [10]:
def best_feature(data, labels):
    scores = {feature: feature_gscore(data, labels, feature) for feature in data.columns}
    best = min(scores, key=scores.get)
    return best, scores

4. Розрахунок критерію Джіні та вибір найкращої ознаки

> - Викликаємо функцію best_feature(T_str, y_str), яка: обчислює значення критерію Джіні для кожної ознаки; визначає ознаку з найменшим критерієм Джіні; зберігає назву цієї ознаки в змінну best, а всі значення критерію Джіні - у змінну scores.

In [11]:
best, scores = best_feature(T_str, y_str)

> - Сортуємо ознаки за зростанням критерію Джіні.

In [12]:
sorted_scores = sorted(scores.items(), key=lambda x: x[1])

> - Виводимо результати на екран.

In [13]:
print("G(x_h) scores ranking:")
for feat, sc in sorted_scores:
    print(f"{feat:6s}: {sc:.6f}")

print("\nBest feature: ", best)

G(x_h) scores ranking:
Sex   : 0.333365
Pclass: 0.418391
SibSp : 0.453226
Parch : 0.458188

Best feature:  Sex


> - Отримуємо деталі для найкращої ознаки.

In [14]:
print("Details:")
details = feature_details(T_str, y_str, best)
for val, info in details.items():
    print(f"  {best} = {val}: passengers count={info['count']}, gini={info['gini']:.6f}, class distribution={info['class_dist']}")

Details:
  Sex = male: passengers count=577, gini=0.306444, class distribution={'0': 468, '1': 109}
  Sex = female: passengers count=314, gini=0.382835, class distribution={'1': 233, '0': 81}


##### **Аналіз результатів:**

Після програмного обчислення коефіцієнт Джіні для таких ознак: Pclass, Sex, SibSp, Parch - бачимо, що найнижче значення цього показнику отримала ознака Sex (0.333365). Тож, відповідно до результату роботи програми, можемо стверджувати, що для розподілення вибірки за цільовою змінною Survived найбільшу інформативність має ознака з найменшим показником - Sex.

Детальний аналіз розподілу ознаки Sex показав суттєву різницю у виживанні між 2 групами (male, female), що підтверджує високу дискримінативну здатність цієї ознаки.

##### **Висновки:**

- Для прогнозування цільової змінної важливо обирати найбільш інформативні ознаки.

- Критерій Джіні дозволяє визначити, яка ознака найкраще розділяє об’єкти за класами.

- У дослідженому датасеті ознака Sex виявилась найбільш інформативною, тобто найкраще відображає, хто вижив, а хто ні.

- Такий підхід допомагає ефективно відбирати ознаки перед побудовою моделей машинного навчання.
